In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# numpy and pandas for data manipulation
import numpy as np
import pandas as pd 

# sklearn preprocessing for dealing with categorical variables
from sklearn.preprocessing import LabelEncoder

# File system manangement
import os

# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')

# matplotlib and seaborn for plotting
import matplotlib.pyplot as plt
import seaborn as sns

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
print(os.listdir("../input/gdsc-nu-datathon-jusan-bank-case-competition"))


app_train = pd.read_csv('../input/gdsc-nu-datathon-jusan-bank-case-competition/train_data.csv')
print('Training data shape: ', app_train.shape)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/gdsc-nu-datathon-jusan-bank-case-competition/sample_res.csv
/kaggle/input/gdsc-nu-datathon-jusan-bank-case-competition/train_data.csv
/kaggle/input/gdsc-nu-datathon-jusan-bank-case-competition/test_data.csv
/kaggle/input/gdsc-nu-datathon-jusan-bank-case-competition/info.json
['sample_res.csv', 'train_data.csv', 'test_data.csv', 'info.json']
Training data shape:  (941531, 48)


In [73]:
def missing_values_table(df):
        # Total missing values
        mis_val = df.isnull().sum()
        
        # Percentage of missing values
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        # Make a table with the results
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        
        # Rename the columns
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        # Sort the table by percentage of missing descending
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        
        # Print some summary information
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        
        # Return the dataframe with missing information
        return mis_val_table_ren_columns

In [74]:
missing_values = missing_values_table(app_train)
missing_values.head(50)

Your selected dataframe has 48 columns.
There are 46 columns that have missing values.


,Missing Values,% of Total Values
transactions_amount_9,941531,100.0
transfers_count_out_8,941531,100.0
transfers_amount_out_8,941531,100.0
transfers_count_in_8,941531,100.0
transfers_amount_in_8,941531,100.0
transactions_count_9,941531,100.0
transactions_count_10,941531,100.0
transactions_amount_10,941531,100.0
date_close,909202,96.6
deal_sum_kzt_product,863225,91.7


In [ ]:
corr_matrix = app_train.corr().abs()
corr_matrix.head()

In [ ]:
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
upper.head()

In [ ]:
app_train[~app_train['transfers_amount_in_9'].isna()].label.value_counts(normalize=True)

In [3]:
correlations = app_train.corr()['client_days'].sort_values()

# Display correlations
print('Most Positive Correlations:\n', correlations.tail(20))

NameError: name 'app_train' is not defined

In [ ]:
all_notifs = app_train.groupby('client_id', as_index=False)['client_id'].count()
print(all_notifs.head())

In [ ]:
(app_train['age_days'] / 365).describe()


In [ ]:
app_train['age_days'].plot.hist(title = 'age_days');
plt.xlabel('age_days');

In [ ]:
correlations = app_train.corr()['deposit_sum_KZT'].sort_values()

# Display correlations
print('Most Positive Correlations:\n', correlations.tail(20))

In [ ]:
app_train.select_dtypes('float').apply(pd.Series.nunique, axis = 0)


In [ ]:
app_train['client_days'].plot.hist(title = 'salary_9 Histogram');
plt.xlabel('cl days');




In [ ]:
app_train['client_days'].describe()


In [ ]:
# Make a new dataframe for polynomial features
poly_features = app_train[['deal_sum_kzt_express', 'client_days', 'transfers_count_in_9', 'DAYS_BIRTH', 'label']]

# imputer for handling missing values
from sklearn.preprocessing import Imputer
imputer = Imputer(strategy = 'median')

poly_target = poly_features['label']

poly_features = poly_features.drop(columns = ['label'])

# Need to impute missing values
poly_features = imputer.fit_transform(poly_features)
poly_features_test = imputer.transform(poly_features_test)

from sklearn.preprocessing import PolynomialFeatures
                                  
# Create the polynomial object with specified degree
poly_transformer = PolynomialFeatures(degree = 3)


# Feature engineering


In [9]:
class Jusan:
    def __init__(self, mode = "train"):
        train_path = "/kaggle/input/gdsc-nu-datathon-jusan-bank-case-competition/train_data.csv"
        test_path = "/kaggle/input/gdsc-nu-datathon-jusan-bank-case-competition/test_data.csv"
        self.data = pd.read_csv(train_path if mode == "train" else test_path)
        self.transformers = self._get_transformers(mode)

        
    def _get_transformers(self, mode="train"):
        def _drop_cliend_id(dframe):
            dframe.drop(labels="client_id",axis=1, inplace=True)
            return dframe
        def _drop_majority_nan_cols(dframe):
            majority_nan_cols = ['date_close', 'deposit_sum_KZT', 'deposit_sum_USD', 
     'deposit_sum_EUR',
     'deal_sum_kzt_express',
     'expired_day_express',
     'deal_start_date_express',
     'plan_end_date_express',
     'deal_sum_kzt_product',
     'expired_day_product',
     'deal_start_date_product',
     'plan_end_date_product',
     'transfers_amount_in_9',
     'transfers_count_in_9',
     'transfers_amount_out_9',
     'transfers_count_out_9',
     'transactions_amount_9',
     'transactions_count_9',
     'salary_9',
     'transfers_amount_in_10',
     'transfers_count_in_10',
     'transfers_amount_out_10',
     'transfers_count_out_10',
     'transactions_amount_10',
     'transactions_count_10',
     'salary_10',
     'transfers_amount_in_8',
     'transfers_count_in_8',
     'transfers_amount_out_8',
     'transfers_count_out_8',
     'transactions_amount_8',
     'transactions_count_8',
     'salary_8']
            dframe.drop(labels=majority_nan_cols, axis=1, inplace = True)
            return dframe
        def _drop_majority_nan_row(dframe):
            rows_majority_nan = (dframe.isna().sum(1) / dframe.shape[1]) > 0.5
            dframe.drop(labels = dframe[rows_majority_nan].index, axis = 0, inplace=True)
            return dframe
        def sum_entities_auth(dframe):
            columns = ['auth_count_8', 'auth_count_9', 'auth_count_10']
            dframe['total_auths'] = dframe[columns].sum(axis =1, inplace = true )
            dframe.drop(columns, axis = 0, inplace = true)
            return dframe
        def sum_entities_transfers_amount_out_(dframe):
            columns = ['transfers_amount_out_8', 'transfers_amount_out_9', 'transfers_amount_out_10']
            dframe['transfers_amount_out_total'] = dframe[columns].sum(axis =1 )
            dframe.drop(columns, axis = 0, inplace = true)

            return dframe    
        def sum_entities_transfers_amount_in_(dframe):
            columns = ['transfers_amount_in_8', 'transfers_amount_in_9', 'transfers_amount_in_10']
            dframe['transfers_amount_in_total'] = dframe[columns].sum(axis =1 )
            dframe.drop(columns, axis = 0, inplace = true)

            return dframe 
        def sum_entities_transactions_amount_out(dframe):
            columns = ['transactions_amount_out_8', 'transactions_amount_out_9', 'transactions_amount_out_10']
            dframe['transactions_amount_total'] = dframe[columns].sum(axis =1 )
            dframe.drop(columns, axis = 0, inplace = true)

            return dframe 
        def sum_entities_salary_X(dframe):
            columns = ['salary_8', 'salary_9', 'salary_10']
            dframe['salary_X'] = dframe[columns].sum(axis =1 )
            dframe.drop(columns, axis = 0, inplace = true)
            return dframe 
        
        def _fill_nan_cols(*cols):
            def _fill_nan_cols_wrapper(dframe):
                for col in cols:
                    dframe[col] = dframe[col].fillna(0)
                return dframe
            return _fill_nan_cols_wrapper
        def _normalize_age_days(dframe):
            age_mean, age_std = 13390.0, 4777.0
            
            dframe["ln_age_days"] = np.log(dframe["age_days"]+1.0)/10
            dframe["ln_age_days"] = dframe["ln_age_days"].fillna(0)
            
            dframe["age_days"] = dframe["age_days"].fillna(age_mean)
            dframe["age_days"] = (dframe["age_days"] - age_mean) / age_std
            
            return dframe
        def _ln_client_days(dframe):
            mean, std = 945.0, 966.0
            dframe["ln_client_days"] = dframe["client_days"].fillna(mean)
            dframe["ln_client_days"] = np.log(dframe["ln_client_days"]+1.0)/10
            dframe.drop(labels="client_days", axis = 1, inplace=True)
            return dframe
        def _modify_country_citizenship(dframe):
            dframe["country"] = dframe["country"].fillna(0.0)
            is_country_kaz = dframe["country"] == 0.0
            dframe.loc[is_country_kaz,"country"] = 1.0
            dframe.loc[~is_country_kaz,"country"] = 0.0
            
            dframe["citizenship"] = dframe["citizenship"].fillna(0.0)
            is_citizenship_kaz = dframe["citizenship"] == 0.0
            dframe.loc[is_citizenship_kaz, "citizenship"] = 1.0
            dframe.loc[~is_citizenship_kaz, "citizenship"] = 0.0
            return dframe
        def _fill_nan_with_means(*cols):
            def _fill_nan_with_means_wrapper(dframe):
                for col in cols:
                    dframe[col] = dframe[col].fillna(dframe[col].mean())
                return dframe
            return _fill_nan_with_means_wrapper
        
        def _fill_nan_with_modes(*cols):
            def _fill_nan_with_modes_wrapper(dframe):
                for col in cols:
                    mode = dframe[col].mode()
                    dframe[col] = dframe[col].fillna(int(mode))
                return dframe
            return _fill_nan_with_modes_wrapper
        def _cato_codes_to_regions(dframe):
            def to_region(val):
                if np.isnan(val):
                    return val
                if val in (1.0, 2.0): #cities
                    return 0.0
                if val in (4.0, 6.0, 7.0, 12.0): #west
                    return 1.0
                if val in (16.0, 19.0, 18.0): #east
                    return 2.0
                if val in (3.0, 9.0, 20.0): #center
                    return 3.0
                if val in (5.0, 8.0, 11.0, 13.0, 17.0): #south
                    return 4.0
                if val in (10.0, 14.0, 15.0): #north
                    return 5.0
            dframe["cato_code"] = dframe["cato_code"].apply(to_region)
            return dframe
        def _opsos_code_shrink(dframe):
            def to_simple(val):
                if val in (8, 12, 5, 3, 10):
                    return val
                return 0
            #change
          
            dframe["opsos_code"] = dframe["opsos_code"].apply(to_simple)
            return dframe
        def _make_feat_product_sum(dframe):
            dframe["product_and_sum_not_null"] = (~dframe["plan_end_date_product"].isna() & ~dframe["deal_sum_kzt_product"].isna()).astype(float)
            return dframe
        def _make_feat_loyal_client(dframe):
            dframe["loyal_client"] = (~dframe["plan_end_date_product"].isna() & ~dframe["age_days"].isna()).astype(float)
            return dframe
        def _make_feat_client_days_ratio(dframe):
            dframe["client_days_ratio"] = (dframe["client_days"] - dframe["client_days"].min()) / dframe["age_days"]
            dframe["client_days_ratio"] = dframe["client_days_ratio"].fillna(0)
            return dframe
        def _make_feat_product_and_closeness_of_next_loan(dframe):
            dframe["product_and_closeness_of_next_loan"] = pd.to_datetime(dframe.plan_end_date_product, yearfirst=True) - pd.to_datetime("2022-11-19", yearfirst=True)
            dframe["product_and_closeness_of_next_loan"] = np.maximum(0, dframe["product_and_closeness_of_next_loan"].dt.days)
            dframe["product_and_closeness_of_next_loan"] = np.log(dframe["product_and_closeness_of_next_loan"]+1.0)/10
            dframe["product_and_closeness_of_next_loan"] = dframe["product_and_closeness_of_next_loan"].fillna(0)
            return dframe
        def _make_feat_expired_date(dframe):
            dframe["ln_exprired_date"] = 1/ np.log(dframe["expired_day_product"]+2.71)
            dframe["ln_exprired_date"] = dframe["ln_exprired_date"].fillna(0.0)
            return dframe
        def _make_dummy_cols(*cols):
            def _make_dummy_cols_wrapper(dframe):
                for col in cols:
                    dframe = pd.concat([dframe.drop(col, axis=1), pd.get_dummies(dframe[col], prefix=col)], axis=1)
#                     dframe.drop(col, inplace=True)
                return dframe
            return _make_dummy_cols_wrapper
        def _to_binary(*cols):
            def _to_binary_wrapper(dframe):
                for col in cols:
                    dframe["has_"+col] = (~dframe[col].isna()).astype(float)
                    if col not in ("monthly_installment"):
                        dframe.drop(labels=col, axis=1,inplace=True)
                return dframe
            return _to_binary_wrapper

        
        
        train_transformers = [_drop_cliend_id,sum_entities_salary_X, sum_entities_transactions_amount_out, sum_entities_transfers_amount_in_,
                              sum_entities_transfers_amount_out_,sum_entities_auth,
                              _make_feat_product_sum, _make_feat_loyal_client,_make_feat_client_days_ratio,
                              _make_feat_product_and_closeness_of_next_loan, _make_feat_expired_date,
                              _to_binary("date_close","deal_sum_kzt_product","plan_end_date_product", "deal_start_date_product",  "deposit_sum_KZT", 
             "deposit_sum_USD", "deposit_sum_EUR" , "plan_end_date_express", "deal_start_date_express", "expired_day_express",
              "monthly_installment"),
#                               _drop_majority_nan_cols,
                              _drop_majority_nan_row, 
                              _fill_nan_cols("gender"), _normalize_age_days,_ln_client_days, _modify_country_citizenship, \
                              _fill_nan_with_means("monthly_installment"),
                               _opsos_code_shrink,
                              _fill_nan_with_modes("opsos_code", "phone_digits", "max_dup", "cato_code",  "auth_count_9", "auth_count_8", "auth_count_10"),\
                              _make_dummy_cols("opsos_code", "phone_digits", "max_dup", "cato_code")
                             ]
        test_transformers = [
                              _make_feat_product_sum, _make_feat_loyal_client,_make_feat_client_days_ratio,
                             sum_entities_salary_X, sum_entities_transactions_amount_out, sum_entities_transfers_amount_in_,
                              sum_entities_transfers_amount_out_,sum_entities_auth,
                              _make_feat_product_and_closeness_of_next_loan, _make_feat_expired_date,
                             _to_binary("date_close","deal_sum_kzt_product","plan_end_date_product", "deal_start_date_product",  "deposit_sum_KZT", 
            "deposit_sum_USD", "deposit_sum_EUR" , "plan_end_date_express", "deal_start_date_express", "expired_day_express",
            "monthly_installment"),
                              #_drop_majority_nan_cols,

                             _fill_nan_cols("gender"), _normalize_age_days,_ln_client_days, _modify_country_citizenship, \
                              _fill_nan_with_means("monthly_installment"),
                               _opsos_code_shrink,
                              _fill_nan_with_modes("opsos_code", "phone_digits", "max_dup", "cato_code",  "auth_count_9", "auth_count_8", "auth_count_10"),\
                              _make_dummy_cols("opsos_code", "phone_digits", "max_dup", "cato_code")
                             ]
        return train_transformers if mode=="train" else test_transformers

    def transform(self):
        for transform in self.transformers:
            print(transform.__name__)
            self.data = transform(self.data)
        return self.data


In [10]:

trainy = Jusan("train").transform()
testy = Jusan("test").transform()


_drop_cliend_id
sum_entities_salary_X


NameError: name 'true' is not defined

In [52]:
testy.shape


(463739, 86)

# Sampling

In [76]:
normal_class = trainy.query('label == 0')
notif_class = trainy.query('label == 1')
print(notif_class.shape)

normal_class = normal_class.sample(frac=1,random_state=1234)
notif_class = notif_class.sample(frac=1,random_state=1234)
resampled = normal_class.sample(n=int(len(notif_class)*40), random_state=1234)


train = pd.concat([notif_class,resampled])


(12332, 87)


In [77]:
normal_class1 = trainy.query('label == 0')
notif_class1 = trainy.query('label == 1')
print(notif_class1.shape)

normal_class1 = normal_class1.sample(frac=1,random_state=1234)
notif_class1 = notif_class1.sample(frac = 1,random_state=1234)
resampled1 = normal_class1.sample(n=int(len(notif_class1)*40), random_state=1234)


train1 = pd.concat([notif_class1,resampled1])

(12332, 87)


In [74]:
from sklearn.metrics import f1_score
from sklearn.metrics import fbeta_score



def lgb_f1_score(y_hat, data):
    y_true = data.get_label()
    y_hat = np.where(y_hat < 0.5, 0, 1)   # scikits f1 doesn't like probabilities
    return 'f1', f1_score(y_true, y_hat), True

In [ ]:
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from sklearn.metrics import precision_score, recall_score, classification_report
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression



SEARCH_PARAMS = {'learning_rate': 0.10,
                'max_depth': 31,
                'num_leaves': 60,
                'feature_fraction': 0.5,
                 "bagging_fraction":0.5,
                }

FIXED_PARAMS={'objective': 'binary',
             'metric': 'binary_error',
            'subsample': 0.2, 
             'boosting':'gbdt',
             'num_boost_round':300,
             'early_stopping_rounds':30}


SEARCH_PARAMS1 = {'learning_rate': 0.07,
                'max_depth': 20,
                'num_leaves': 32,
                'feature_fraction': 0.6,
                 "bagging_fraction":0.6,
                }

FIXED_PARAMS1={'objective': 'binary',
             'metric': 'binary_error',
                'subsample': 0.2, 
             'boosting':'gbdt',
             'num_boost_round':300,
             'early_stopping_rounds':34}


for c in train.columns:
    col_type = train[c].dtype
    if col_type == 'object' or col_type.name == 'category':
        train[c] = train[c].astype('category')
                           
                           
X = train.drop(['label'], axis=1)
y = train['label']        
X1 = train1.drop(['label'], axis=1)
y1 = train1['label']  
important_features = np.zeros(X.shape[1])
                    
def train_evaluate(search_params):


   
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=1234)
    X_train1, X_valid1, y_train1, y_valid1 = train_test_split(X1, y1, test_size=0.2, random_state=1234)


    train_data = lgb.Dataset(X_train, label=y_train)
    train_data1 = lgb.Dataset(X_train1, label=y_train1)

    valid_data = lgb.Dataset(X_valid, label=y_valid, reference=train_data)
    valid_data1 = lgb.Dataset(X_valid1, label=y_valid1, reference=train_data1)


    params = {'metric':FIXED_PARAMS['metric'],
                 'objective':FIXED_PARAMS['objective'],
                 **search_params}
    evals_result = {}

    model = lgb.train(params, train_data,
                         valid_sets=[valid_data],
                         num_boost_round=FIXED_PARAMS['num_boost_round'],
                         early_stopping_rounds=FIXED_PARAMS['early_stopping_rounds'],
                         valid_names=['valid'],feval = lgb_f1_score, evals_result=evals_result)
    evals_result1 = {}

    model1 = lgb.train(params, train_data1,
                         valid_sets=[valid_data1],
                         num_boost_round=FIXED_PARAMS['num_boost_round'],
                         early_stopping_rounds=FIXED_PARAMS['early_stopping_rounds'],
                         valid_names=['valid'],feval = lgb_f1_score, evals_result=evals_result1)
    
    lgb.plot_metric(evals_result, metric='f1')
    yy = trainy['label']        
    Xx = trainy.drop(['label'], axis = 1)
    predicted_label = model.predict(Xx)
    predicted_label1 = model1.predict(Xx)   
    

    print(f1_score(((predicted_label*.44+predicted_label1*.6)>.56).astype(int), yy))
    

    return model, model1
    
    
model, model1 = train_evaluate(SEARCH_PARAMS)
lgb.plot_importance(model, figsize=(10,10))




[LightGBM] [Info] Number of positive: 9883, number of negative: 394606
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.060799 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 5784
[LightGBM] [Info] Number of data points in the train set: 404489, number of used features: 76
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.024433 -> initscore=-3.687072
[LightGBM] [Info] Start training from score -3.687072
[1]	valid's binary_error: 0.024218	valid's f1: 0
Training until validation scores don't improve for 30 rounds
[2]	valid's binary_error: 0.0235357	valid's f1: 0.0673981
[3]	valid's binary_error: 0.0233181	valid's f1: 0.0881671
[4]	valid's binary_error: 0.0230116	valid's f1: 0.116888
[5]	valid's binary_error: 0.0226852	valid's f1: 0.146577
[6]	valid's binary_error: 0.0225567	valid's f1: 0.161089
[7]	valid's binary_error: 0.0225072	valid


    predicted_label = model.predict(Xx)
    predicted_label1 = model1.predict(Xx)
    
    
    val_pred1=model.predict(X_train)
    test_pred1=model.predict(X_valid)
    val_pred1=pd.DataFrame(val_pred1)
    test_pred1=pd.DataFrame(test_pred1)


    val_pred2=model1.predict(X_train)
    test_pred2=model1.predict(X_valid)
    val_pred2=pd.DataFrame(val_pred2)
    test_pred2=pd.DataFrame(test_pred2)
    

    df_val=pd.concat([X_train, val_pred1,val_pred2],axis=1)
    df_test=pd.concat([X_valid, test_pred1,test_pred2],axis=1)

    modelX = LogisticRegression()
    modelX.fit(df_val,y_valid)
    model.score(df_test,y_valid)

In [31]:
X = train.drop(['client_id', 'label'], axis=1)
y = train['label']         

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=1234)
print(y_valid.shape)

(13579,)


# > The result


In [ ]:
categoricals =app_train.select_dtypes('object').columns.to_numpy()
print(categoricals)


In [ ]:
%%timeit
model.predict(X)



In [ ]:
X.shape